# Patagonia trip in python - map with pictures and diary

In [128]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import os
# define here some variables:
ROOT_GPX_FILES = '/Users/filippomichelon/Documents/PersonalCode/garmin_patagonia_gpx_files'


In [ ]:
# define here the function to extract .gpx files from a folder and extract it
import gpxpy
import gpxpy.gpx
import pandas as pd
import os
from typing import List

def extract_gpx_files_from_folder(folder_path: str) -> pd.DataFrame:
    """
    Extracts multiple .gpx files from a specified folder and combines their data into a Pandas DataFrame.
    
    Parameters:
    folder_path (str): Path to the folder containing .gpx files.
    
    Returns:
    pd.DataFrame: DataFrame containing latitude, longitude, elevation, time, and file name.
    """
    data = []
    file_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.gpx')]
    
    for file_path in file_paths:
        with open(file_path, 'r') as gpx_file:
            gpx = gpxpy.parse(gpx_file)
            for track in gpx.tracks:
                for segment in track.segments:
                    for point in segment.points:
                        data.append({
                            'latitude': point.latitude,
                            'longitude': point.longitude,
                            'elevation': point.elevation,
                            'time': point.time,
                            'file': os.path.basename(file_path)
                        })
    
    return pd.DataFrame(data)

df = extract_gpx_files_from_folder(ROOT_GPX_FILES)
df

In [ ]:

for day in df['file'].unique():
    day_track = df[df['file'] == day]

In [148]:
import pandas as pd
import requests

def generate_meteo_url(latitude: float, longitude: float, date: str, hour: str) -> str:
    """
    Generates the Open-Meteo API URL with the specified parameters for latitude, longitude, date, and hour.
    """
    base_url = "https://api.open-meteo.com/v1/forecast"
    url = f"{base_url}?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,precipitation,wind_speed_10m,winddirection_10m&start_date={date}&end_date={date}&hourly={hour}&format=csv"
    return url

def get_weather_for_time(df: pd.DataFrame) -> pd.DataFrame:
    """
    Gets the weather data for each time in df.time and returns a DataFrame with the weather info.
    
    Parameters:
    df (pd.DataFrame): DataFrame with 'latitude', 'longitude', and 'time' columns containing coordinates and timestamps.
    
    Returns:
    pd.DataFrame: A DataFrame containing weather data corresponding to each time in df.
    """
    weather_data = []
    
    for _, row in df.iterrows():
        latitude = row['latitude']
        longitude = row['longitude']
        timestamp = row['time']
        
        # Extract date and hour from the timestamp
        date = timestamp.strftime('%Y-%m-%d')
        hour = timestamp.strftime('%H')
        
        # Generate the URL for the Open-Meteo API based on the date and hour
        url = generate_meteo_url(latitude, longitude, date, hour)
        
        # Request the data from the API
        response = requests.get(url)
        
        if response.status_code == 200:
            # Parse the CSV response into a DataFrame
            meteo_df = pd.read_csv(pd.compat.StringIO(response.text), skiprows=6)  # Skip the header rows
            # Extract the weather data for that time
            weather_info = meteo_df[meteo_df['time'] == timestamp.strftime('%Y-%m-%dT%H:%M:%SZ')].iloc[0]
            weather_info['latitude'] = latitude
            weather_info['longitude'] = longitude
            weather_info['time'] = timestamp
            weather_data.append(weather_info)
        else:
            print(f"Error fetching data for {timestamp}: {response.status_code}")
    
    # Return the weather data as a DataFrame
    return pd.DataFrame(weather_data)

# Example usage:
# Assuming df is your DataFrame with 'latitude', 'longitude', and 'time' columns
#weather_df = get_weather_for_time(df)
#print(weather_df)


In [153]:
weather_df = get_weather_for_time(df.tail(3))
print(weather_df)

Error fetching data for 2025-02-23 17:38:15+00:00: 400
Error fetching data for 2025-02-23 17:38:16+00:00: 400
Error fetching data for 2025-02-23 17:38:17+00:00: 400
Empty DataFrame
Columns: []
Index: []


In [82]:
#LUIGI'S FUNCTION 
# Take the meteo dataset using the function below
def get_meteo_dataset():
    """Get the meteo dataset from the open-meteo API.
    Note how easy it is to get data from the web with pandas! As long as we give the URL of the csv data, pandas can read it.
    """
    np.random.seed(42)
    URL = "https://api.open-meteo.com/v1/forecast?latitude=52.52&longitude=13.41&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,precipitation,wind_speed_10m,winddirection_10m&start_date=2025-03-01&end_date=2025-03-10&format=csv"
    df = pd.read_csv(URL, skiprows=6)  # read the csv file, skipping the first 3 rows (a header)
    df.columns = [col.split(" ")[0] for col in df.columns]  # simplify column names
    df["time"] = pd.to_datetime(df["time"])  # convert the time column to datetime
    df["hour"], df["dayofyear"] = df["time"].dt.hour, df["time"].dt.dayofyear  # extract the hour and day of year
    df["weekdays"] = df["time"].dt.day_name()  # extract the day of the week

    return df

meteo_df = get_meteo_dataset()